In [5]:
import tensorflow as tf
import QGOpt as qgo
import math

#---------------------------------------------------------------------------------#
ham_dim = 20  # dimension of a hamiltonian
renorm_ham_dim = 10  # dimension of a renormalized hamiltonian
number_of_steps = tf.constant(200, dtype=tf.int32)  # number of optimization steps
#---------------------------------------------------------------------------------#

# hamiltonian generation
herm = qgo.manifolds.HermitianMatrix()
h = herm.random((ham_dim, ham_dim), dtype=tf.complex128)

# complex Stiefel manifold
stiefel = qgo.manifolds.StiefelManifold()

# initial random isometric matrix
q = stiefel.random((ham_dim, renorm_ham_dim), dtype=tf.complex128)
q = qgo.manifolds.complex_to_real(q)
q = tf.Variable(q)

# optimizers
opts = {'GD':qgo.optimizers.RSGD(stiefel, 0.05),
        'momentum_GD':qgo.optimizers.RSGD(stiefel, 0.1, 0.9),
        'Nesterov_momentum_GD':qgo.optimizers.RSGD(stiefel, 0.1, 0.9, use_nesterov=True),
        'Adam':qgo.optimizers.RAdam(stiefel, 0.2),
        'AmsGrad':qgo.optimizers.RAdam(stiefel, 0.2, ams=True)}

# exact solution of the problem
exact_solution = tf.math.real(tf.reduce_sum(tf.linalg.eigvalsh(h)[:renorm_ham_dim]))

# optimization function
def optimize(q, h, number_of_steps):
    i = tf.constant(0, dtype=tf.int32)
    loss = tf.constant(0, dtype=tf.float64)
    def body(i, loss):
        with tf.GradientTape() as tape:
            qc = qgo.manifolds.real_to_complex(q)
            loss = tf.math.real(tf.linalg.trace(tf.linalg.adjoint(qc) @ h @ qc))
        grad = tape.gradient(loss, q)
        opt.apply_gradients(zip([grad], [q]))
        return i + 1, loss
    cond = lambda i, loss: i < number_of_steps
    _, loss = tf.while_loop(cond, body, [i, loss])
    return loss

# optimization loops
err_dict = {}
for key, opt in opts.items():
    loss = optimize(q, h, number_of_steps)
    err_dict[key] = tf.math.abs(loss - exact_solution)
err_dict

{'GD': <tf.Tensor: id=231111, shape=(), dtype=float64, numpy=2.8450585887185298e-08>,
 'momentum_GD': <tf.Tensor: id=266346, shape=(), dtype=float64, numpy=7.105427357601002e-15>,
 'Nesterov_momentum_GD': <tf.Tensor: id=302381, shape=(), dtype=float64, numpy=7.105427357601002e-15>,
 'Adam': <tf.Tensor: id=347832, shape=(), dtype=float64, numpy=6.73736622047727e-11>,
 'AmsGrad': <tf.Tensor: id=398692, shape=(), dtype=float64, numpy=1.7215029401995707e-10>}